In [13]:
import pandas as pd
import numpy as np

In [14]:
df = pd.read_csv("../../data/collision_data.csv")
df = df.drop_duplicates(subset=["C_CASE"])

df["C_MNTH_sin"] = np.sin(2*np.pi * df["C_MNTH"] / 12)
df["C_MNTH_cos"] = np.cos(2*np.pi * df["C_MNTH"] / 12)

df["C_WDAY_sin"] = np.sin(2*np.pi * df["C_WDAY"] / 7)
df["C_WDAY_cos"] = np.cos(2*np.pi * df["C_WDAY"] / 7)

df["C_HOUR_sin"] = np.sin(2*np.pi * df["C_HOUR"] / 24)
df["C_HOUR_cos"] = np.cos(2*np.pi * df["C_HOUR"] / 24)

df.head()

,C_YEAR,C_MNTH,C_WDAY,C_HOUR,C_SEV,C_VEHS,C_CONF,C_RCFG,C_WTHR,C_RSUR,C_RALN,C_TRAF,C_CASE,C_MNTH_sin,C_MNTH_cos,C_WDAY_sin,C_WDAY_cos,C_HOUR_sin,C_HOUR_cos
0,2005,1.0,1.0,11.0,2,1.0,4.0,2.0,4.0,5.0,3.0,NaN,915642,0.5,0.866025,0.781831,0.62349,0.258819,-0.965926
1,2005,1.0,1.0,15.0,2,2.0,2.0,3.0,1.0,3.0,1.0,18.0,915794,0.5,0.866025,0.781831,0.62349,-0.707107,-0.707107
2,2005,1.0,1.0,13.0,2,2.0,35.0,2.0,1.0,1.0,1.0,3.0,915805,0.5,0.866025,0.781831,0.62349,-0.258819,-0.965926
3,2005,1.0,1.0,13.0,2,1.0,4.0,NaN,4.0,4.0,3.0,NaN,915877,0.5,0.866025,0.781831,0.62349,-0.258819,-0.965926
4,2005,1.0,1.0,20.0,2,1.0,2.0,NaN,1.0,3.0,3.0,NaN,915919,0.5,0.866025,0.781831,0.62349,-0.866025,0.500000


In [15]:
numeric_features = [
    "C_YEAR", 
    "C_MNTH_sin", "C_MNTH_cos",
    "C_WDAY_sin", "C_WDAY_cos",
    "C_HOUR_sin", "C_HOUR_cos"
]

categorical_features = [
    "C_RCFG",
    "C_WTHR",
]

df = df[numeric_features + categorical_features]
df = df.dropna()

group_cols = numeric_features + categorical_features

df_grouped = df.groupby(group_cols).size().reset_index(name="collision_count")
df_grouped.head()

,C_YEAR,C_MNTH_sin,C_MNTH_cos,C_WDAY_sin,C_WDAY_cos,C_HOUR_sin,C_HOUR_cos,C_RCFG,C_WTHR,collision_count
0,2005,-1.0,-1.836970e-16,-0.974928,-0.222521,-1.0,-1.836970e-16,1.0,1.0,28
1,2005,-1.0,-1.836970e-16,-0.974928,-0.222521,-1.0,-1.836970e-16,1.0,2.0,5
2,2005,-1.0,-1.836970e-16,-0.974928,-0.222521,-1.0,-1.836970e-16,1.0,3.0,8
3,2005,-1.0,-1.836970e-16,-0.974928,-0.222521,-1.0,-1.836970e-16,1.0,6.0,1
4,2005,-1.0,-1.836970e-16,-0.974928,-0.222521,-1.0,-1.836970e-16,2.0,1.0,53


In [16]:
df_grouped.collision_count.describe()

count    276698.000000
mean          6.171989
std           9.297658
min           1.000000
25%           1.000000
50%           2.000000
75%           6.000000
max         104.000000
Name: collision_count, dtype: float64

In [17]:
from sklearn.model_selection import train_test_split

X = df_grouped[numeric_features + categorical_features]
y = df_grouped["collision_count"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


## Poisson regression

In [18]:
from sklearn.linear_model import PoissonRegressor
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline

preprocessor = ColumnTransformer(
    transformers=[
        ("num", "passthrough", numeric_features),
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features)
    ]
)

model = Pipeline([
    ("preprocess", preprocessor),
    ("poisson", PoissonRegressor(alpha=1.0))
])

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

c:\Users\jylis\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_glm\glm.py:286: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result(


In [19]:
poisson = model.named_steps["poisson"]
intercept = poisson.intercept_
coefs = poisson.coef_
feature_names = model.named_steps["preprocess"].get_feature_names_out()

coef_table = pd.DataFrame({
    "feature": feature_names,
    "coefficient": coefs
})
coef_table 

,feature,coefficient
0,num__C_YEAR,-0.000319
1,num__C_MNTH_sin,-0.130937
2,num__C_MNTH_cos,-0.089107
3,num__C_WDAY_sin,-0.021612
4,num__C_WDAY_cos,-0.076314
5,num__C_HOUR_sin,-0.251449
6,num__C_HOUR_cos,-0.337399
7,cat__C_RCFG_1.0,0.341615
8,cat__C_RCFG_2.0,0.447878
9,cat__C_RCFG_3.0,-0.311216


In [20]:
from sklearn.metrics import (
    mean_absolute_error,
    mean_squared_error,
    r2_score,
    mean_poisson_deviance,
    mean_gamma_deviance
)
import numpy as np

print("MAE:", mean_absolute_error(y_test, y_pred))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))
print("R2:", r2_score(y_test, y_pred))
print("Poisson Deviance:", mean_poisson_deviance(y_test, y_pred))
print("Gamma Deviance:", mean_gamma_deviance(y_test, y_pred))

MAE: 4.165332910208762
RMSE: 6.7998913087634465
R2: 0.4683526659004852
Poisson Deviance: 4.0267806921258
Gamma Deviance: 0.7432194645674522


## Negative binomial regression

In [21]:
import statsmodels.api as sm

model_nb2 = sm.NegativeBinomial(y_train, X_train)
results_nb2 = model_nb2.fit()

print(results_nb2.summary())

c:\Users\jylis\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\discrete\discrete_model.py:3663: RuntimeWarning: overflow encountered in exp
  return np.exp(linpred)
c:\Users\jylis\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\discrete\discrete_model.py:3379: RuntimeWarning: divide by zero encountered in log
  llf = coeff + size*np.log(prob) + endog*np.log(1-prob)
c:\Users\jylis\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\discrete\discrete_model.py:3470: RuntimeWarning: invalid value encountered in multiply
  dparams = exog*a1 * (y-mu)/(mu+a1)
c:\Users\jylis\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\discrete\discrete_model.py:3470: RuntimeWarning: invalid value encountered in divide
  dparams = exog*a1 * (y-mu)/(mu+a1)
c:\Users\jylis\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\discrete\discrete_model.py:3473: RuntimeWarning: invalid value encountered in divide
  

Optimization terminated successfully.
         Current function value: 2.530707
         Iterations: 18
         Function evaluations: 36
         Gradient evaluations: 31
                     NegativeBinomial Regression Results                      
Dep. Variable:        collision_count   No. Observations:               221358
Model:               NegativeBinomial   Df Residuals:                   221349
Method:                           MLE   Df Model:                            8
Date:                Wed, 03 Dec 2025   Pseudo R-squ.:                  0.1252
Time:                        19:42:40   Log-Likelihood:            -5.6019e+05
converged:                       True   LL-Null:                   -6.4040e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
C_YEAR         0.0017   2.54e-06    67

In [22]:
y_pred = results_nb2.predict(X_test)

print("MAE:", mean_absolute_error(y_test, y_pred))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))
print("R2:", r2_score(y_test, y_pred))
print("Poisson Deviance:", mean_poisson_deviance(y_test, y_pred))

MAE: 4.1169325198136475
RMSE: 7.195225400665381
R2: 0.4047375253739146
Poisson Deviance: 4.308623930812018
